<a href="https://colab.research.google.com/github/iRaez/applied_datascience_capstone/blob/main/capstone_mod3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Applied Data Science Capstone Project


---





## Module 3 Assignment Part 1

<h2><b>Setup</b></h2>
<h3>Import Libraries</h3>

In [197]:
import pandas as pd
import numpy as np

<h3><b>Load Data</b></h3>


In [35]:
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
df = pd.read_html(url, header = 0)
df

[    Postal Code  ...                                      Neighbourhood
 0           M1A  ...                                       Not assigned
 1           M2A  ...                                       Not assigned
 2           M3A  ...                                          Parkwoods
 3           M4A  ...                                   Victoria Village
 4           M5A  ...                          Regent Park, Harbourfront
 ..          ...  ...                                                ...
 175         M5Z  ...                                       Not assigned
 176         M6Z  ...                                       Not assigned
 177         M7Z  ...                                       Not assigned
 178         M8Z  ...  Mimico NW, The Queensway West, South of Bloor,...
 179         M9Z  ...                                       Not assigned
 
 [180 rows x 3 columns],
                                           Unnamed: 0  ... Unnamed: 17
 0  NL NS PE NB QC ON MB S

Check the dataframe to see how many tables there are in the webpage:

In [36]:
len(df)

3

Based on the website, table 1 is the table we are looking for.

In [17]:
df[0]

,Postal Code,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
...,...,...,...
175,M5Z,Not assigned,Not assigned
176,M6Z,Not assigned,Not assigned
177,M7Z,Not assigned,Not assigned
178,M8Z,Etobicoke,"Mimico NW, The Queensway West, South of Bloor,..."


Assign this into a new dataframe and let's call it `dfcan`.

In [105]:
dfcan = df[0].assign()

In [51]:
df[0].shape

(180, 3)

In [52]:
dfcan.shape

(103, 3)

### Data Cleaning

Assignment requirements:


*   The dataframe will consist of three columns: PostalCode, Borough, and Neighborhood

*   Only process the cells that have an assigned borough. Ignore cells with a borough that is **Not assigned**.

*   More than one neighborhood can exist in one postal code area. For example, in the table on the Wikipedia page, you will notice that **M5A** is listed twice and has two neighborhoods: **Harbourfront** and **Regent Park**. These two rows will be combined into one row with the neighborhoods separated with a comma as shown in **row 11**  in the above table.

*   If a cell has a borough but a **Not assigned**  neighborhood, then the neighborhood will be the same as the borough.





In [106]:
# making boolean series for borough 
filter1 = dfcan["Borough"]!="Not assigned"

# making boolean series for neighbourhood 
filter2 = dfcan["Neighbourhood"]!="Not assigned"

# filtering data on basis of both filters 
dfcan.where(filter1 & filter2, inplace = True) 

# display 
dfcan.head()


,Postal Code,Borough,Neighbourhood
0,NaN,NaN,NaN
1,NaN,NaN,NaN
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


The `dataframe.where()` method filters the dataframe according to the defined conditions and replaces all values with `NaN` if it doesn't meet the conditions. So next what we will do is use the `.dropna()` method to remove all missing values.

More information from the pandas documentation can be found [here](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.dropna.html)

In [107]:
# First check how many none NaN values exist
dfcan.count(0)

Postal Code      103
Borough          103
Neighbourhood    103
dtype: int64

In [108]:
# Next we will drop all rows where all values are NaN
dfcan.dropna(axis=0, how='all', inplace=True)
dfcan.reset_index(drop = True, inplace=True)
dfcan

,Postal Code,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
...,...,...,...
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North"
99,M4Y,Downtown Toronto,Church and Wellesley
100,M7Y,East Toronto,"Business reply mail Processing Centre, South C..."
101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu..."


### Last cell for notebook below

In [109]:
dfcan.shape

(103, 3)

## Module 3 Assignment Part 2

<h2><b>Setup</b></h2>
<h3>Import Libraries</h3>
For this section geocoder is required

###### Install geocoder

In [76]:
! pip install geocoder

     |████████████████████████████████| 102kB 3.9MB/s 


### Import Geocoder

In [77]:
import geocoder

## Get geographical data
Get latitude and longitude data for each postal code and assign it to the `dfcan` dataframe

### Using Geocoder Package

In [ ]:
# Get first postal code from dataframe to get lat_lng data
postcode = dfcan['Postal Code']

#Create lists to store latitude and longitude data
lat = []
lng = []

i = 0

#iterate each postal code and pass it into geocoder
for postal_code in postcode:
  print('Current Postal Code', postal_code)
  # initialize variable to None
  lat_lng_coords = None

  # loop until you get the coordinates
  while (lat_lng_coords is None):
    print('requesting...')
    g = geocoder.osm('{}, Toronto, Ontario'.format(postal_code))
    lat_lng_coords = g.latlng
    if lat_lng_coords is None :
      print('We got nothing, trying again...')
      continue
    print('Got it!')

  #assign value into latitude and longitude
  latitude = lat_lng_coords[0]
  longitude = lat_lng_coords[1]
  i +=1
  
  #Put into list
  lat[i-1]=latitude
  lng[i-1]=longitude

#once For loop is done
dfcan['Latitude'] = lat.values
dfcan['Longitude'] = lng.values

###Note: unable to get above section to work using the geocoder package. Following section will use the provided coordinates csv file.

In [126]:
#import csv file into dataframe
geodf = pd.read_csv('https://cocl.us/Geospatial_data')
geodf

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476
...,...,...,...
98,M9N,43.706876,-79.518188
99,M9P,43.696319,-79.532242
100,M9R,43.688905,-79.554724
101,M9V,43.739416,-79.588437


In order to query the geodf, convert the index to the postal code.

In [141]:
geodf.set_index('Postal Code', inplace=True) #using inplace=True to not create a new dataframe
geodf.head()

,Latitude,Longitude
Postal Code,,
M1B,43.806686,-79.194353
M1C,43.784535,-79.160497
M1E,43.763573,-79.188711
M1G,43.770992,-79.216917
M1H,43.773136,-79.239476


In [ ]:
#Map Latitude and longitude data to corresponding Postal code
postcode = dfcan['Postal Code'] #assigning postal code column from original dataframe

#creating empty list to store coordinates
lat=np.zeros(len(dfcan))
lng=np.zeros(len(dfcan))
i = 0

for canPost in postcode :
  i+=1
  lat[i-1] = geodf.loc[canPost][0]
  lng[i-1] = geodf.loc[canPost][1]


In [203]:
dfcan['Latitude']=lat
dfcan['Longitude']=lng
dfcan.head(10)

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
5,M9A,Etobicoke,"Islington Avenue, Humber Valley Village",43.667856,-79.532242
6,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
7,M3B,North York,Don Mills,43.745906,-79.352188
8,M4B,East York,"Parkview Hill, Woodbine Gardens",43.706397,-79.309937
9,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937
